# Package import

In [26]:
from math import cos, sin, tan
import numpy as np

from transforms3d import affines as a
#from transforms3d import taitbryan as tb
#from transforms3d import quaternions as q
from transforms3d import _gohlketransforms as t

# 基本的なユーティリティ関数

## ベクトルの扱い

ベクトルの定義

In [4]:
def vec2(x, y): return np.array([x, y], dtype=np.float32)
def vec3(x, y, z): return np.array([x, y, z], dtype=np.float32)
def vec4(x, y, z, w): return np.array([x, y, z, w], dtype=np.float32)
def vec(*args): return np.array(args, dtype=np.float32)

if __name__ == '__main__':
    v2 = vec2(1, 2)
    v3 = vec3(1, 2, 3)
    v4 = vec4(1, 2, 3, 4)
    v8 = vec(1, 2, 3, 4, 5, 6, 7, 8)
    print('v2 =', v2)
    print('v3 =', v3)
    print('v4 =', v4)
    print('v8 =', v8)

v2 = [ 1.  2.]
v3 = [ 1.  2.  3.]
v4 = [ 1.  2.  3.  4.]
v8 = [ 1.  2.  3.  4.  5.  6.  7.  8.]


ベクトルの正規化

In [5]:
def normalize(v):
    norm=np.linalg.norm(v)
    if norm==0: return v
    return v/norm

if __name__ == '__main__':
    def __normalize(vname, v):
        nv = normalize(v)
        print('|normalize({0})| = {1}'.format(vname, np.linalg.norm(nv)))
    __normalize('v2', v2)
    __normalize('v3', v3)
    __normalize('v4', v4)
    __normalize('v8', v8)

|normalize(v2)| = 0.9999999403953552
|normalize(v3)| = 0.9999999403953552
|normalize(v4)| = 0.9999999403953552
|normalize(v8)| = 1.0


## Matrices

4x4行列はrow-major表現で与えと、column-majorなものを与える。`mat4x4`関数の最後で転置行列をとっていることに注意。

In [6]:
def mat4x4(m00, m01, m02, m03, m10, m11, m12, m13,
        m20, m21, m22, m23, m30, m31, m32, m33):
    return np.array([m00, m01, m02, m03, m10, m11, m12, m13,
        m20, m21, m22, m23, m30, m31, m32, m33], dtype=np.float32).reshape((4, 4))

def tmat4x4(m00, m01, m02, m03, m10, m11, m12, m13,
        m20, m21, m22, m23, m30, m31, m32, m33):
    return mat4x4(m00, m01, m02, m03, m10, m11, m12, m13,
        m20, m21, m22, m23, m30, m31, m32, m33).transpose()

identity = t.identity_matrix
#id4x4 = identity(4)

if __name__ == '__main__':
    print('mat4x4(1, 2, 3, ...):\n{0}'.format(mat4x4(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)))
    print('tmat4x4(1, 2, 3, ...):\n{0}'.format(tmat4x4(1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16)))


    cat = t.concatenate_matrices

mat4x4(1, 2, 3, ...):
[[  1.   2.   3.   4.]
 [  5.   6.   7.   8.]
 [  9.  10.  11.  12.]
 [ 13.  14.  15.  16.]]
tmat4x4(1, 2, 3, ...):
[[  1.   5.   9.  13.]
 [  2.   6.  10.  14.]
 [  3.   7.  11.  15.]
 [  4.   8.  12.  16.]]


In [15]:
NoTranslation = [0, 0, 0]
NoRotation = [[1, 0, 0], [0, 1, 0], [0, 0, 1]]
NoZoom = [1, 1, 1]

def compose(T=NoTranslation, R=NoRotation, Z=NoZoom): return a.compose(T, R, Z)

def scale(x, y, z):
    return mat4x4(
        x, 0, 0, 0,
        0, y, 0, 0,
        0, 0, z, 0,
        0, 0, 0, 1)

# def rotate(radian, dir): return t.rotation_matrix(radian, dir)
def rotate(radian, x, y, z):
    c = cos(radian); s = sin(radian)
    t = vec3((1-c)*x, (1-c)*y, (1-c)*z)
    return mat4x4(
        c + t[0]*x, 0 + t[0]*y + s*z, 0 + t[0]*z - s*y, 0,
        0 + t[1]*x - s*z, c + t[1]*y, 0 + t[1]*z + s*x, 0,
        0 + t[2]*x + s*y, 0 + t[2]*y - s*x, c + t[2]*z, 0,
        0, 0, 0, 0)

def translate(x, y, z):
    return mat4x4(
    1, 0, 0, 0,
    0, 1, 0, 0,
    0, 0, 1, 0,
    x, y, z, 1)

shear = t.shear_matrix

if __name__ == '__main__':
    np.set_printoptions(precision=3, suppress=True)
    print('Scale(2, 3, 4):\n{0}'.format(scale(2, 3, 4)))
    print('Rotate[X:90]\n{0}'.format(rotate(np.pi/3, 1, 0, 0)))
    print('Rotate[Y:90]\n{0}'.format(rotate(np.pi/3, 0, 1, 0)))
    print('Rotate[Z:90]\n{0}'.format(rotate(np.pi/3, 0, 0, 1)))
    print('Translate(1, 2, 3):\n{0}'.format(translate(1, 2, 3)))

Scale(2, 3, 4):
[[ 2.  0.  0.  0.]
 [ 0.  3.  0.  0.]
 [ 0.  0.  4.  0.]
 [ 0.  0.  0.  1.]]
Rotate[X:90]
[[ 1.     0.     0.     0.   ]
 [ 0.     0.5    0.866  0.   ]
 [ 0.    -0.866  0.5    0.   ]
 [ 0.     0.     0.     0.   ]]
Rotate[Y:90]
[[ 0.5    0.    -0.866  0.   ]
 [ 0.     1.     0.     0.   ]
 [ 0.866  0.     0.5    0.   ]
 [ 0.     0.     0.     0.   ]]
Rotate[Z:90]
[[ 0.5    0.866  0.     0.   ]
 [-0.866  0.5    0.     0.   ]
 [ 0.     0.     1.     0.   ]
 [ 0.     0.     0.     0.   ]]
Translate(1, 2, 3):
[[ 1.  0.  0.  0.]
 [ 0.  1.  0.  0.]
 [ 0.  0.  1.  0.]
 [ 1.  2.  3.  1.]]


## 空間変換のための関数群

### 視野変換 (LookAt)

$$\begin {align}
F &= \mathit {normalize}(\mathit {center} - \mathit {eye}) \\
S &= \mathit {normalize}(F \times \mathit {up}) \\
H &= S \times F \\
\mathit {LookAt} &= \begin {pmatrix}
  S_0 & H_0 & -F_0 & 0 \\
  S_1 & H_1 & -F_1 & 0 \\
  S_2 & H_2 & -F_2 & 0 \\
  -S \cdot \mathit {eye} & -H \cdot \mathit {eye} & F \cdot \mathit {eye} & 0
\end {pmatrix}
\end {align}$$

In [19]:
def lookat(eye, center, up):
    # Forward, Side, and Head
    F = normalize(center - eye)
    S = normalize(np.cross(F, up))
    U = np.cross(S, F)
    
    return mat4x4(
        S[0], S[1], S[2], -np.dot(S, eye),
        U[0], U[1], U[2], -np.dot(U, eye),
        -F[0], -F[1], -F[2], np.dot(F, eye),
        0, 0, 0, 1)

    return mat4x4(
        S[0], U[0], -F[0], 0,
        S[1], U[1], -F[1], 0,
        S[2], U[2], -F[2], 0,
        -np.dot(S, eye), -np.dot(U, eye), np.dot(F, eye), 1)

'''# GLM implementation
template <typename T, precision P> GLM_FUNC_QUALIFIER tmat4x4<T, P>
lookAt (tvec3<T, P> const & eye, tvec3<T, P> const & center, tvec3<T, P> const & up) {
    tvec3<T, P> const f(normalize(center - eye));
    tvec3<T, P> const s(normalize(cross(f, up)));
    tvec3<T, P> const u(cross(s, f));

    tmat4x4<T, P> Result(1);
    Result[0][0] = s.x; Result[1][0] = s.y; Result[2][0] = s.z;
    Result[0][1] = u.x; Result[1][1] = u.y; Result[2][1] = u.z;
    Result[0][2] =-f.x; Result[1][2] =-f.y; Result[2][2] =-f.z;
    Result[3][0] =-dot(s, eye); Result[3][1] =-dot(u, eye); Result[3][2] = dot(f, eye);
    return Result;
}'''

if __name__ == '__main__':
    eye    = vec3(5, 0, -5) # 視点
    center = vec3(5, 0, 0)  # 投影面上の点：XY平面を正面から眺める
    up     = vec3(0, 1, 0)  # Y軸方向を頭の向きとする
    LookAt = lookat(eye, center, up)
    print('LookAt:\n{0}'.format(LookAt))
    
    points = vec([0, 0, -2.5, 1,
                  0, 0, 0, 1,
                  0, 0, 5, 1,
                  0, 0, 45, 1]).reshape((-1, 4)).T
    print('LookAt 変換によって4つの点\n{0}\nは、それぞれ次の点に移動します\n{1}'
          .format(points, LookAt.dot(points)))

LookAt:
[[-1.  0.  0.  5.]
 [ 0.  1. -0. -0.]
 [-0. -0. -1. -5.]
 [ 0.  0.  0.  1.]]
LookAt 変換によって4つの点
[[  0.    0.    0.    0. ]
 [  0.    0.    0.    0. ]
 [ -2.5   0.    5.   45. ]
 [  1.    1.    1.    1. ]]
は、それぞれ次の点に移動します
[[  5.    5.    5.    5. ]
 [  0.    0.    0.    0. ]
 [ -2.5  -5.  -10.  -50. ]
 [  1.    1.    1.    1. ]]


## Ortho transformation

In [31]:
def ortho(left, right, bottom, top, zNear=-1, zFar=1):
    rl, tb, fn = right - left, top - bottom, zFar - zNear
    return mat4x4(
            2/rl, 0, 0, -(right + left)/rl,
            0, 2/tb, 0, -(top + bottom)/tb,
            0, 0, -2/fn, -(zFar + zNear)/fn,
            0, 0, 0, 1)

''' GLM Implementation

template <typename T> GLM_FUNC_QUALIFIER tmat4x4<T, defaultp>
ortho (T left, T right, T bottom, T top, T zNear, T zFar) {
    tmat4x4<T, defaultp> Result(1);
    Result[0][0] = static_cast<T>(2) / (right - left);
    Result[1][1] = static_cast<T>(2) / (top - bottom);
    Result[2][2] = - static_cast<T>(2) / (zFar - zNear);
    Result[3][0] = - (right + left) / (right - left);
    Result[3][1] = - (top + bottom) / (top - bottom);
    Result[3][2] = - (zFar + zNear) / (zFar - zNear);
    return Result;
}

template <typename T> GLM_FUNC_QUALIFIER tmat4x4<T, defaultp>
ortho (T left, T right, T bottom, T top) {
    tmat4x4<T, defaultp> Result(1);
    Result[0][0] = static_cast<T>(2) / (right - left);
    Result[1][1] = static_cast<T>(2) / (top - bottom);
    Result[2][2] = - static_cast<T>(1);
    Result[3][0] = - (right + left) / (right - left);
    Result[3][1] = - (top + bottom) / (top - bottom);
    return Result;
}'''

if __name__ == '__main__':
    print('Ortho(0, 10, 0, 10, 2, 12):\n{0}'.format(ortho(0, 10, 0, 10, 2, 12)))

Ortho(0, 10, 0, 10, 2, 12):
[[ 0.2  0.   0.  -1. ]
 [ 0.   0.2  0.  -1. ]
 [ 0.   0.  -0.2 -1.4]
 [ 0.   0.   0.   1. ]]


## Frustum transformation

In [38]:
def frustum(left, right, bottom, top, near, far):
    rl, tb, fn = right - left, top - bottom, far - near
    return mat4x4(
            2 * near / rl, 0, (right + left) / rl, 0,
            0, 2 * near / tb, (top + bottom) / tb, 0,
            0, 0, -(far + near) / fn, -2 * far * near / fn,
            0, 0, 0, 0)

''' GLM implementation
template <typename T> GLM_FUNC_QUALIFIER tmat4x4<T, defaultp>
frustum (T left, T right, T bottom, T top, T nearVal, T farVal) {
    tmat4x4<T, defaultp> Result(0);
    Result[0][0] = (static_cast<T>(2) * nearVal) / (right - left);
    Result[1][1] = (static_cast<T>(2) * nearVal) / (top - bottom);
    Result[2][0] = (right + left) / (right - left);
    Result[2][1] = (top + bottom) / (top - bottom);
    Result[2][2] = -(farVal + nearVal) / (farVal - nearVal);
    Result[2][3] = static_cast<T>(-1);
    Result[3][2] = -(static_cast<T>(2) * farVal * nearVal) / (farVal - nearVal);
    return Result;
}
'''

if __name__ == '__main__':
    print('Frustum(0, 10, 0, 10, 2, 12):\n{0}'.format(frustum(0, 10, 0, 10, 2, 12)))

Frustum(0, 10, 0, 10, 2, 12):
[[ 0.4  0.   1.   0. ]
 [ 0.   0.4  1.   0. ]
 [ 0.   0.  -1.4 -4.8]
 [ 0.   0.   0.   0. ]]


In [35]:
def perspective(fovy, aspect, near, far):
    tFovy2 = tan(fovy / 2)
    return mat4x4(
            1 / (aspect * tFovy2), 0, 0, 0,
            0, 1 / tFovy2, 0, 0,
            0, 0, -(far + near) / (far - near), -2 * far * near / (far - near),
            0, 0, -1, 0)

#def perspective(fovy, aspect, near, far):
#    s = 1 / tan(fovy / 2 * np.pi / 180)
#    M = np.diag([s, s, - float(far + near) / float(far - near), 0])
#    M[2][3] = -1; M[3][2] = - 2 * far * near / float(far - near)
#    return M.astype(np.float32)

''' GLM implementation
template <typename T> GLM_FUNC_QUALIFIER tmat4x4<T, defaultp>
perspective (T fovy, T aspect, T zNear, T zFar) {
    assert(abs(aspect - std::numeric_limits<T>::epsilon()) > static_cast<T>(0));

    T const tanHalfFovy = tan(fovy / static_cast<T>(2));

    tmat4x4<T, defaultp> Result(static_cast<T>(0));
    Result[0][0] = static_cast<T>(1) / (aspect * tanHalfFovy);
    Result[1][1] = static_cast<T>(1) / (tanHalfFovy);
    Result[2][2] = - (zFar + zNear) / (zFar - zNear);
    Result[2][3] = - static_cast<T>(1);
    Result[3][2] = - (static_cast<T>(2) * zFar * zNear) / (zFar - zNear);
    return Result;
}'''


if __name__ == '__main__':
    print('Perspective(np.pi / 3, 1, 2, 12):\n{0}'.format(perspective(np.pi / 3, 1, 2, 12)))

Perspective(np.pi / 3, 1, 2, 12):
[[ 1.732  0.     0.     0.   ]
 [ 0.     1.732  0.     0.   ]
 [ 0.     0.    -1.4   -4.8  ]
 [ 0.     0.    -1.     0.   ]]


In [44]:
def perspectiveFov(fov, width, height, near, far):
    h = cos(fov/2) / sin(fov/2)
    w = h * height / width
    return mat4x4(
            w, 0, 0, 0,
            0, h, 0, 0,
            0, 0, -(far + near) / (far - near), -1, 
            0, 0, -2 * far * near / (far - near))

'''GLM implementation
template <typename T> GLM_FUNC_QUALIFIER tmat4x4<T, defaultp>
perspectiveFov (T fov, T width, T height, T zNear, T zFar) {
    assert(width > static_cast<T>(0));
    assert(height > static_cast<T>(0));
    assert(fov > static_cast<T>(0));

    T const rad = fov;
    T const h = glm::cos(static_cast<T>(0.5) * rad) / glm::sin(static_cast<T>(0.5) * rad);
    T const w = h * height / width; ///todo max(width , Height) / min(width , Height)?

    tmat4x4<T, defaultp> Result(static_cast<T>(0));
    Result[0][0] = w;
    Result[1][1] = h;
    Result[2][2] = - (zFar + zNear) / (zFar - zNear);
    Result[2][3] = - static_cast<T>(1);
    Result[3][2] = - (static_cast<T>(2) * zFar * zNear) / (zFar - zNear);
    return Result;
}'''

if __name__ == '__main__':
    pass

In [45]:
def project(obj, Model, Proj, viewport):
    V = Proj.dot(Model.dot(obj))
    V = V / V[3] / 2 + 0.5
    V[0] = V[0] * viewport[2] + viewport[0]
    V[1] = V[1] * viewport[3] + viewport[1]
    return V

'''GLM implementation
template <typename T, typename U, precision P> GLM_FUNC_QUALIFIER tvec3<T, P>
project (tvec3<T, P> const & obj, tmat4x4<T, P> const & model, tmat4x4<T, P> const & proj, tvec4<U, P> const & viewport) {
    tvec4<T, P> tmp = tvec4<T, P>(obj, T(1));
    tmp = model * tmp;
    tmp = proj * tmp;

    tmp /= tmp.w;
    tmp = tmp * T(0.5) + T(0.5);
    tmp[0] = tmp[0] * T(viewport[2]) + T(viewport[0]);
    tmp[1] = tmp[1] * T(viewport[3]) + T(viewport[1]);

    return tvec3<T, P>(tmp);
}'''

if __name__ == '__main__':
#    print('Project(obj, Model, Proj, viewport):\n{0}'.format(perspective(np.pi / 3, 1, 2, 12)))
    pass